Metric for importance assesment
-Zonation 2
-Zonation 3
-Zonation 6

Source data variable description
https://www.metsakeskus.fi/sites/default/files/document/mvjaete-koodisto-ja-tietokantakuvaus_0.xlsx


The Finnish Forest Center forest stand classes
Luokat Suomen Metsäkeskuksen metsävarakuvioista
fertilityclass	1	Lehto, letto ja lehtomainen suo (ja ruohoturvekangas)
fertilityclass	2	Lehtomainen kangas, vastaava suo ja ruohoturvekangas
fertilityclass	3	Tuore kangas, vastaava suo ja mustikkaturvekangas
fertilityclass	4	Kuivahko kangas, vastaava suo ja puolukkaturvekangas
fertilityclass	5	Kuiva kangas, vastaava suo ja varputurvekangas
fertilityclass	6	Karukkokangas, vastaava suo (ja jäkäläturvekangas)
fertilityclass	7	Kalliomaa ja hietikko
fertilityclass	8	Lakimetsä ja tunturi

treestand
697	type	1	Inventointi-tyyppi
697	type	2	Laskenta-tyyppi.
697	type	3	Ennuste-tyyppi
(type 3 jätetty pois ja otettu korkein arvo luokasta 1 ja 2)

join standid to treestandsummary by treestandsummaryid
join meanage from treestandsummary by (added) standid

Ageclasses
-40 young (age_class = 1)
-40-140 middle (age_class = 2)
-over 140 old (age_class = 3)

Region of interest
-3 maakuntaa. Pirkanmaa, Kainuu ja Uusimaa
downloaded 12.2.2024 from https://avoin.metsakeskus.fi/aineistot/Metsavarakuviot/Maakunta/

Thresholds
-Zonation-arvo > zonation.percentile(0.7) best 30 %
-Zonation-arvo > zonation.percentile(0.9) best 10 %

# Zonation1

## Protection distribution

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import dask_geopandas as dask_gpd
from scipy.spatial import cKDTree
from rasterstats import zonal_stats
import rasterio
from scipy import stats
import fiona

#to see numbers as non scientific
np.set_printoptions(suppress=True)

filenameZonation = r"YuorPathToFileWithZonationValues"

filename = r"YourPath\MV_Kainuu.gpkg"


#stand polygons
stand = dask_gpd.read_file(filenameZonation, npartitions=2)
stand = stand.compute()
print("stand",len(stand))
#standids to add age etc
treestand = dask_gpd.read_file(filename, layer='treestand', npartitions=2)
treestand = treestand.compute()
condition = treestand.type.isin([2,1])
treestand = treestand[condition]
print("treestand",len(treestand))
#wanted data to add
treestandsummary = dask_gpd.read_file(filename,layer='treestandsummary', npartitions=2)
treestandsummary = treestandsummary.compute()
print("treestandsummary",len(treestandsummary))
#wanted data to add
specialfeature = dask_gpd.read_file(filename,layer='specialfeature', npartitions=2)
specialfeature = specialfeature.compute()
print("specialfeature",len(treestandsummary))
columns = ['standid', 'featurecode']
featurecode = specialfeature[columns]
featurecode = featurecode.groupby('standid').min()
featurecode.reset_index(inplace=True)
 

#add standid to treestandsummaries and join to stand polygons based on that
joined_layer = treestandsummary.merge(treestand, left_on='treestandid', right_on='treestandid', how='left')
joined_layer = stand.merge(joined_layer, left_on='standid', right_on='standid', how='left')
len(joined_layer)

#select the columns to add+join key
columns = ['standid', 'meanage']
age = joined_layer[columns]

#one polygon includes many rows of subtable. add aggregation method, here its max
#select the larger one of inventointi- and laskenta-tyyppi
max_age = age.groupby('standid').max()
max_age.reset_index(inplace=True)
#print(max_age)

#add the desired column
stand_age = stand.merge(max_age, left_on='standid', right_on='standid', how='left')
print("number of empty ages",stand_age.meanage.isna().sum())
stand_age = stand_age.merge(featurecode, left_on='standid', right_on='standid', how='left')
print(len(stand_age))

#divide into age classes
conditions = [
    (stand_age['meanage'] < 40),
    (stand_age['meanage'] >= 40) & (stand_age['meanage'] <= 140),
    (stand_age['meanage'] > 140)
]
choices = [1,2,3]
stand_age['age_class'] = np.select(conditions, choices, default=0) #0 when nodata


out_file = r"YoutPath"
combined = dask_gpd.read_parquet(out_file, npartitions=2)
combined = combined.compute()

joined = gpd.sjoin(stand_age, combined)
# Count the number of points in each polygon. use an unique id column to group
print(f"groupingvariable (standid) is unique {stand_age.standid.is_unique}")
counts = joined.groupby('standid').size()

# Convert the counts Series to a DataFrame
counts_df = counts.reset_index()
counts_df.columns = ['standid', 'count']

# Merge the counts with the original GeoDataFrame
stand_age = pd.merge(stand_age, counts_df, on='standid', how='left')

# Fill NaN values with 0 (assuming no overlap means a count of 0)
stand_age['protected'] = stand_age['count'].fillna(0)

#divide into protected classes
stand_age['protection_class']  = np.where(stand_age['protected']  > 0, 1, 0)


COI = [ 'maingroup', 'subgroup', 'fertilityclass',
       'soiltype', 'drainagestate', 'ditchingyear', 'thinningyear',
       'developmentclass', 'standquality', 'maintreespecies', 'area',
       'areadecrease', 'creationtime', 'updatetime', 'Zonation_mean', 'Zonation_median',
        'Zonation_min', 'Zonation_max',
        'meanage', 'protection_class','age_class']

print("number of Nulls")
print(stand_age[COI].isna().sum())

COI_continuous_sum = ['fertilityclass','area']
condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result = grouped.sum()
result = result.rename(columns={'area': 'Area_Protected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result2 = grouped.sum()
result2 = result2.rename(columns={'area': 'Area_UnProtected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'Area_Protected30'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result4 = grouped.sum()
result4 = result4.rename(columns={'area': 'Area_UnProtected30'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result4, left_on='fertilityclass', right_index=True)


# As percentages
percentage_df = merged_df.apply(lambda x: x / x.sum() * 100)
percentage_df = percentage_df.rename(columns={'Area_Protected10': 'AP10_perc', 'Area_UnProtected10': 'AU10_perc', 'Area_Protected30': 'AP30_perc', 'Area_UnProtected30': 'AU30_perc'})

# combne with areas
merged_df2 = merged_df.merge(percentage_df, left_on='fertilityclass', right_index=True)

merged_df2['ProtPerc10'] = merged_df2.apply(lambda row: (row['Area_Protected10'] / (row['Area_UnProtected10']+row['Area_Protected10'])) * 100, axis=1)
merged_df2['ProtPerc30'] = merged_df2.apply(lambda row: (row['Area_Protected30'] / (row['Area_UnProtected30']+row['Area_Protected30'])) * 100, axis=1)
order = ['Area_Protected10', 'Area_UnProtected10', 'ProtPerc10','Area_Protected30','Area_UnProtected30','ProtPerc30']

Suojeltu10 = merged_df2.Area_Protected10.sum()/merged_df2.Area_UnProtected10.sum()*100
Suojeltu30 = merged_df2.Area_Protected30.sum()/merged_df2.Area_UnProtected30.sum()*100

print(f"Zonation raja-arvo top 10%:lle on {stand_age.Zonation_mean.quantile(0.9)}")
print(f"Zonation raja-arvo top 30%:lle on {stand_age.Zonation_mean.quantile(0.7)}")
print(f"Suojellun osuus parhaasta 10 %:sta on {Suojeltu10} %")
print(f"Suojellun osuus parhaasta 30 %:sta on {Suojeltu30} %")

merged_df2[order]

stand 658767
treestand 1285332
treestandsummary 1307908
specialfeature 1307908
number of empty ages 9143
658767
groupingvariable (standid) is unique True
number of Nulls
maingroup                0
subgroup                43
fertilityclass          52
soiltype                52
drainagestate           43
ditchingyear        547281
thinningyear        658767
developmentclass     68719
standquality        658767
maintreespecies       8242
area                     0
areadecrease             0
creationtime             0
updatetime               0
Zonation_mean           70
Zonation_median         70
Zonation_min            70
Zonation_max            70
meanage               9143
protection_class         0
age_class                0
dtype: int64
Zonation raja-arvo top 10%:lle on 0.7491201889723318
Zonation raja-arvo top 30%:lle on 0.5846679065352992
Suojellun osuus parhaasta 10 %:sta on 28.27711943715897 %
Suojellun osuus parhaasta 30 %:sta on 13.299406778987366 %


,Area_Protected10,Area_UnProtected10,ProtPerc10,Area_Protected30,Area_UnProtected30,ProtPerc30
fertilityclass,,,,,,
1.0,516.571,1405.626,26.873988,543.159,1599.432,25.350569
2.0,1532.900,7549.646,16.877426,2379.886,14962.428,13.723001
3.0,14826.484,31331.572,32.121119,20944.933,109815.630,16.017775
4.0,4685.225,28658.974,14.051095,9086.777,109487.463,7.663365
5.0,1194.902,11337.248,9.534693,2880.772,33184.538,7.987654
6.0,35.105,316.528,9.983420,96.759,1057.034,8.386166
7.0,17.601,62.045,22.099038,22.122,239.428,8.458039


## General

In [2]:
#the most occurring (mode) value within categorical variables, COI is ColumnsOfInterest
COI_categorical = [ 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('fertilityclass')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='fertilityclass')

#mean values for continuous variable
COI_continuous = ['fertilityclass','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('fertilityclass')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['fertilityclass','area']
grouped = stand_age[COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)

merged_df


,maingroup,subgroup,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
fertilityclass,,,,,,,,,,,,,,
1.0,1,1.0,20.0,04,2.0,0.793607,0.867747,0.846782,0.675212,0.919543,71.557634,0.218110,3026,2401.454
2.0,1,1.0,20.0,03,2.0,1.005798,0.619279,0.595107,0.448593,0.738706,57.310326,0.114036,27500,27659.447
3.0,1,1.0,10.0,03,1.0,1.547536,0.471074,0.459225,0.357861,0.595446,55.823550,0.070742,284882,440865.137
4.0,1,1.0,10.0,03,1.0,1.626456,0.460689,0.449584,0.350450,0.582181,55.974620,0.059683,257276,418448.155
5.0,1,3.0,60.0,02,1.0,1.723862,0.466095,0.455517,0.350294,0.593144,54.600764,0.082722,80922,139498.326
6.0,3,3.0,60.0,02,1.0,2.667440,0.392356,0.382297,0.289267,0.530366,49.819822,0.062724,4464,11907.452
7.0,2,1.0,50.0,NaN,1.0,0.899645,0.524754,0.509797,0.416053,0.623887,95.918699,0.057543,643,578.472
8.0,2,1.0,30.0,NaN,2.0,2.368000,0.191079,0.191079,0.145297,0.236861,44.000000,0.000000,2,4.736


In [3]:
#the most occurring (mode) value within categorical variables
COI_categorical = ['age_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('age_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='age_class')

#mean values for continuous variable
COI_continuous = ['age_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('age_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['age_class','area']
grouped = stand_age[COI_continuous_sum].groupby('age_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='age_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='age_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
age_class,,,,,,,,,,,,,,,
0,1,1.0,3.0,60.0,A0,1.0,1.782936,0.412042,0.403474,0.302065,0.550267,NaN,0.092967,9143,16301.381
1,1,1.0,3.0,10.0,T2,1.0,1.771272,0.405517,0.396236,0.309610,0.536478,22.374426,0.051375,156458,277129.652
2,1,1.0,3.0,10.0,03,1.0,1.519247,0.496161,0.483394,0.375210,0.616605,65.512456,0.075801,487724,740973.343
3,1,1.0,3.0,10.0,04,1.0,1.323119,0.553273,0.532338,0.406825,0.663655,154.943771,0.088938,5442,7200.416


In [4]:
#the most occurring (mode) value within categorical variables
COI_categorical = ['protection_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('protection_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='protection_class')

#mean values for continuous variable
COI_continuous = ['protection_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage']
grouped = stand_age[COI_continuous].groupby('protection_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['protection_class','area']
grouped = stand_age[COI_continuous_sum].groupby('protection_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='protection_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='protection_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,size,area_sum
protection_class,,,,,,,,,,,,,,
0,1,1.0,3.0,10.0,03,1.0,1.571099,0.465345,0.453039,0.352363,0.587553,55.255368,612425,962180.319
1,1,1.0,3.0,10.0,03,1.0,1.713877,0.587503,0.580283,0.444973,0.722463,64.062319,46342,79424.473


# Zonation2

## Protection distribution

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import dask_geopandas as dask_gpd
from scipy.spatial import cKDTree
from rasterstats import zonal_stats
import rasterio
from scipy import stats
import fiona

#to see numbers as non scientific
np.set_printoptions(suppress=True)

filenameZonation = r"YuorPathToFileWithZonationValues"

filename = r"YourPath\MV_Kainuu.gpkg"


#stand polygons
stand = dask_gpd.read_file(filenameZonation, npartitions=2)
stand = stand.compute()
print("stand",len(stand))
#standids to add age etc
treestand = dask_gpd.read_file(filename, layer='treestand', npartitions=2)
treestand = treestand.compute()
condition = treestand.type.isin([2,1])
treestand = treestand[condition]
print("treestand",len(treestand))
#wanted data to add
treestandsummary = dask_gpd.read_file(filename,layer='treestandsummary', npartitions=2)
treestandsummary = treestandsummary.compute()
print("treestandsummary",len(treestandsummary))
#wanted data to add
specialfeature = dask_gpd.read_file(filename,layer='specialfeature', npartitions=2)
specialfeature = specialfeature.compute()
print("specialfeature",len(treestandsummary))
columns = ['standid', 'featurecode']
featurecode = specialfeature[columns]
featurecode = featurecode.groupby('standid').min()
featurecode.reset_index(inplace=True)
 

#add standid to treestandsummaries and join to stand polygons based on that
joined_layer = treestandsummary.merge(treestand, left_on='treestandid', right_on='treestandid', how='left')
joined_layer = stand.merge(joined_layer, left_on='standid', right_on='standid', how='left')
len(joined_layer)

#select the columns to add+join key
columns = ['standid', 'meanage']
age = joined_layer[columns]

#one polygon includes many rows of subtable. add aggregation method, here its max
#select the larger one of inventointi- and laskenta-tyyppi
max_age = age.groupby('standid').max()
max_age.reset_index(inplace=True)
#print(max_age)

#add the desired column
stand_age = stand.merge(max_age, left_on='standid', right_on='standid', how='left')
print("number of empty ages",stand_age.meanage.isna().sum())
stand_age = stand_age.merge(featurecode, left_on='standid', right_on='standid', how='left')
print(len(stand_age))

#divide into age classes
conditions = [
    (stand_age['meanage'] < 40),
    (stand_age['meanage'] >= 40) & (stand_age['meanage'] <= 140),
    (stand_age['meanage'] > 140)
]
choices = [1,2,3]
stand_age['age_class'] = np.select(conditions, choices, default=0) #0 when nodata


out_file = r"YourPath"
combined = dask_gpd.read_parquet(out_file, npartitions=2)
combined = combined.compute()

joined = gpd.sjoin(stand_age, combined)
# Count the number of points in each polygon. use an unique id column to group
print(f"groupingvariable (standid) is unique {stand_age.standid.is_unique}")
counts = joined.groupby('standid').size()

# Convert the counts Series to a DataFrame
counts_df = counts.reset_index()
counts_df.columns = ['standid', 'count']

# Merge the counts with the original GeoDataFrame
stand_age = pd.merge(stand_age, counts_df, on='standid', how='left')

# Fill NaN values with 0 (assuming no overlap means a count of 0)
stand_age['protected'] = stand_age['count'].fillna(0)

#divide into protected classes
stand_age['protection_class']  = np.where(stand_age['protected']  > 0, 1, 0)


COI = [ 'maingroup', 'subgroup', 'fertilityclass',
       'soiltype', 'drainagestate', 'ditchingyear', 'thinningyear',
       'developmentclass', 'standquality', 'maintreespecies', 'area',
       'areadecrease', 'creationtime', 'updatetime', 'Zonation_mean', 'Zonation_median',
        'Zonation_min', 'Zonation_max',
        'meanage', 'protection_class','age_class']

print("number of Nulls")
print(stand_age[COI].isna().sum())

COI_continuous_sum = ['fertilityclass','area']
condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result = grouped.sum()
result = result.rename(columns={'area': 'Area_Protected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result2 = grouped.sum()
result2 = result2.rename(columns={'area': 'Area_UnProtected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'Area_Protected30'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result4 = grouped.sum()
result4 = result4.rename(columns={'area': 'Area_UnProtected30'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result4, left_on='fertilityclass', right_index=True)


# As percentages
percentage_df = merged_df.apply(lambda x: x / x.sum() * 100)
percentage_df = percentage_df.rename(columns={'Area_Protected10': 'AP10_perc', 'Area_UnProtected10': 'AU10_perc', 'Area_Protected30': 'AP30_perc', 'Area_UnProtected30': 'AU30_perc'})

# combne with areas
merged_df2 = merged_df.merge(percentage_df, left_on='fertilityclass', right_index=True)

merged_df2['ProtPerc10'] = merged_df2.apply(lambda row: (row['Area_Protected10'] / (row['Area_UnProtected10']+row['Area_Protected10'])) * 100, axis=1)
merged_df2['ProtPerc30'] = merged_df2.apply(lambda row: (row['Area_Protected30'] / (row['Area_UnProtected30']+row['Area_Protected30'])) * 100, axis=1)
order = ['Area_Protected10', 'Area_UnProtected10', 'ProtPerc10','Area_Protected30','Area_UnProtected30','ProtPerc30']

Suojeltu10 = merged_df2.Area_Protected10.sum()/merged_df2.Area_UnProtected10.sum()*100
Suojeltu30 = merged_df2.Area_Protected30.sum()/merged_df2.Area_UnProtected30.sum()*100

print(f"Zonation raja-arvo top 10%:lle on {stand_age.Zonation_mean.quantile(0.9)}")
print(f"Zonation raja-arvo top 30%:lle on {stand_age.Zonation_mean.quantile(0.7)}")
print(f"Suojellun osuus parhaasta 10 %:sta on {Suojeltu10} %")
print(f"Suojellun osuus parhaasta 30 %:sta on {Suojeltu30} %")

merged_df2[order]

stand 658767
treestand 1285332
treestandsummary 1307908
specialfeature 1307908
number of empty ages 9143
658767
groupingvariable (standid) is unique True
number of Nulls
maingroup                0
subgroup                43
fertilityclass          52
soiltype                52
drainagestate           43
ditchingyear        547281
thinningyear        658767
developmentclass     68719
standquality        658767
maintreespecies       8242
area                     0
areadecrease             0
creationtime             0
updatetime               0
Zonation_mean           70
Zonation_median         70
Zonation_min            70
Zonation_max            70
meanage               9143
protection_class         0
age_class                0
dtype: int64
Zonation raja-arvo top 10%:lle on 0.7569859576225281
Zonation raja-arvo top 30%:lle on 0.58110258509232
Suojellun osuus parhaasta 10 %:sta on 39.00880077469934 %
Suojellun osuus parhaasta 30 %:sta on 15.93827638368312 %


,Area_Protected10,Area_UnProtected10,ProtPerc10,Area_Protected30,Area_UnProtected30,ProtPerc30
fertilityclass,,,,,,
1.0,514.793,1393.601,26.975195,547.535,1587.792,25.641740
2.0,1405.189,6627.854,17.492611,2227.209,13575.859,14.093523
3.0,17470.554,33050.498,34.580741,23374.806,114790.601,16.917987
4.0,6152.266,21446.994,22.291417,10867.854,96119.460,10.158077
5.0,1809.464,7366.350,19.719929,3766.490,29382.339,11.362362
6.0,40.732,229.681,15.062885,105.968,988.325,9.683695
7.0,11.375,136.792,7.677148,29.043,289.188,9.126389


## General

In [2]:
#the most occurring (mode) value within categorical variables, COI is ColumnsOfInterest
COI_categorical = [ 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('fertilityclass')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='fertilityclass')

#mean values for continuous variable
COI_continuous = ['fertilityclass','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('fertilityclass')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['fertilityclass','area']
grouped = stand_age[COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)

merged_df


,maingroup,subgroup,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
fertilityclass,,,,,,,,,,,,,,
1.0,1,1.0,20.0,04,2.0,0.793607,0.860310,0.837822,0.665391,0.916505,71.557634,0.218110,3026,2401.454
2.0,1,1.0,20.0,03,2.0,1.005798,0.599584,0.577178,0.426164,0.727118,57.310326,0.114036,27500,27659.447
3.0,1,1.0,10.0,03,1.0,1.547536,0.465182,0.453052,0.343926,0.595132,55.823550,0.070742,284882,440865.137
4.0,1,1.0,10.0,03,1.0,1.626456,0.438245,0.427204,0.324635,0.565042,55.974620,0.059683,257276,418448.155
5.0,1,3.0,60.0,02,1.0,1.723862,0.454359,0.443402,0.334022,0.584554,54.600764,0.082722,80922,139498.326
6.0,3,3.0,60.0,02,1.0,2.667440,0.388848,0.377600,0.276034,0.532561,49.819822,0.062724,4464,11907.452
7.0,2,1.0,50.0,NaN,1.0,0.899645,0.552800,0.531054,0.418916,0.665720,95.918699,0.057543,643,578.472
8.0,2,1.0,30.0,NaN,2.0,2.368000,0.347374,0.347374,0.257763,0.436985,44.000000,0.000000,2,4.736


In [3]:
#the most occurring (mode) value within categorical variables
COI_categorical = ['age_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('age_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='age_class')

#mean values for continuous variable
COI_continuous = ['age_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('age_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['age_class','area']
grouped = stand_age[COI_continuous_sum].groupby('age_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='age_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='age_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
age_class,,,,,,,,,,,,,,,
0,1,1.0,3.0,60.0,A0,1.0,1.782936,0.403451,0.395595,0.288633,0.548059,NaN,0.092967,9143,16301.381
1,1,1.0,3.0,10.0,T2,1.0,1.771272,0.368858,0.360526,0.273849,0.505914,22.374426,0.051375,156458,277129.652
2,1,1.0,3.0,10.0,03,1.0,1.519247,0.489518,0.476308,0.360974,0.614947,65.512456,0.075801,487724,740973.343
3,1,1.0,3.0,10.0,04,1.0,1.323119,0.570427,0.549531,0.412518,0.687337,154.943771,0.088938,5442,7200.416


In [4]:
#the most occurring (mode) value within categorical variables
COI_categorical = ['protection_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('protection_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='protection_class')

#mean values for continuous variable
COI_continuous = ['protection_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage']
grouped = stand_age[COI_continuous].groupby('protection_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['protection_class','area']
grouped = stand_age[COI_continuous_sum].groupby('protection_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='protection_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='protection_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,size,area_sum
protection_class,,,,,,,,,,,,,,
0,1,1.0,3.0,10.0,03,1.0,1.571099,0.447361,0.434839,0.328210,0.575929,55.255368,612425,962180.319
1,1,1.0,3.0,10.0,03,1.0,1.713877,0.631800,0.626124,0.491616,0.757787,64.062319,46342,79424.473


# Zonation 3

## Protection distribution

In [6]:
import geopandas as gpd
import pandas as pd
import numpy as np
import dask_geopandas as dask_gpd
from scipy.spatial import cKDTree
from rasterstats import zonal_stats
import rasterio
from scipy import stats
import fiona

#to see numbers as non scientific
np.set_printoptions(suppress=True)

filenameZonation = r"YuorPathToFileWithZonationValues"

filename = r"YourPath\MV_Kainuu.gpkg"

#stand polygons
stand = dask_gpd.read_file(filenameZonation, npartitions=2)
stand = stand.compute()
print("stand",len(stand))
#standids to add age etc
treestand = dask_gpd.read_file(filename, layer='treestand', npartitions=2)
treestand = treestand.compute()
condition = treestand.type.isin([2,1])
treestand = treestand[condition]
print("treestand",len(treestand))
#wanted data to add
treestandsummary = dask_gpd.read_file(filename,layer='treestandsummary', npartitions=2)
treestandsummary = treestandsummary.compute()
print("treestandsummary",len(treestandsummary))
#wanted data to add
specialfeature = dask_gpd.read_file(filename,layer='specialfeature', npartitions=2)
specialfeature = specialfeature.compute()
print("specialfeature",len(treestandsummary))
columns = ['standid', 'featurecode']
featurecode = specialfeature[columns]
featurecode = featurecode.groupby('standid').min()
featurecode.reset_index(inplace=True)
 

#add standid to treestandsummaries and join to stand polygons based on that
joined_layer = treestandsummary.merge(treestand, left_on='treestandid', right_on='treestandid', how='left')
joined_layer = stand.merge(joined_layer, left_on='standid', right_on='standid', how='left')
len(joined_layer)

#select the columns to add+join key
columns = ['standid', 'meanage']
age = joined_layer[columns]

#one polygon includes many rows of subtable. add aggregation method, here its max
#select the larger one of inventointi- and laskenta-tyyppi
max_age = age.groupby('standid').max()
max_age.reset_index(inplace=True)
#print(max_age)

#add the desired column
stand_age = stand.merge(max_age, left_on='standid', right_on='standid', how='left')
print("number of empty ages",stand_age.meanage.isna().sum())
stand_age = stand_age.merge(featurecode, left_on='standid', right_on='standid', how='left')
print(len(stand_age))

#divide into age classes
conditions = [
    (stand_age['meanage'] < 40),
    (stand_age['meanage'] >= 40) & (stand_age['meanage'] <= 140),
    (stand_age['meanage'] > 140)
]
choices = [1,2,3]
stand_age['age_class'] = np.select(conditions, choices, default=0) #0 when nodata


out_file = r"YourPath"
combined = dask_gpd.read_parquet(out_file, npartitions=2)
combined = combined.compute()

joined = gpd.sjoin(stand_age, combined)
# Count the number of points in each polygon. use an unique id column to group
print(f"groupingvariable (standid) is unique {stand_age.standid.is_unique}")
counts = joined.groupby('standid').size()

# Convert the counts Series to a DataFrame
counts_df = counts.reset_index()
counts_df.columns = ['standid', 'count']

# Merge the counts with the original GeoDataFrame
stand_age = pd.merge(stand_age, counts_df, on='standid', how='left')

# Fill NaN values with 0 (assuming no overlap means a count of 0)
stand_age['protected'] = stand_age['count'].fillna(0)

#divide into protected classes
stand_age['protection_class']  = np.where(stand_age['protected']  > 0, 1, 0)


COI = [ 'maingroup', 'subgroup', 'fertilityclass',
       'soiltype', 'drainagestate', 'ditchingyear', 'thinningyear',
       'developmentclass', 'standquality', 'maintreespecies', 'area',
       'areadecrease', 'creationtime', 'updatetime', 'Zonation_mean', 'Zonation_median',
        'Zonation_min', 'Zonation_max',
        'meanage', 'protection_class','age_class']

print("number of Nulls")
print(stand_age[COI].isna().sum())

COI_continuous_sum = ['fertilityclass','area']
condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result = grouped.sum()
result = result.rename(columns={'area': 'Area_Protected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result2 = grouped.sum()
result2 = result2.rename(columns={'area': 'Area_UnProtected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'Area_Protected30'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result4 = grouped.sum()
result4 = result4.rename(columns={'area': 'Area_UnProtected30'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result4, left_on='fertilityclass', right_index=True)


# As percentages
percentage_df = merged_df.apply(lambda x: x / x.sum() * 100)
percentage_df = percentage_df.rename(columns={'Area_Protected10': 'AP10_perc', 'Area_UnProtected10': 'AU10_perc', 'Area_Protected30': 'AP30_perc', 'Area_UnProtected30': 'AU30_perc'})

# combne with areas
merged_df2 = merged_df.merge(percentage_df, left_on='fertilityclass', right_index=True)

merged_df2['ProtPerc10'] = merged_df2.apply(lambda row: (row['Area_Protected10'] / (row['Area_UnProtected10']+row['Area_Protected10'])) * 100, axis=1)
merged_df2['ProtPerc30'] = merged_df2.apply(lambda row: (row['Area_Protected30'] / (row['Area_UnProtected30']+row['Area_Protected30'])) * 100, axis=1)
order = ['Area_Protected10', 'Area_UnProtected10', 'ProtPerc10','Area_Protected30','Area_UnProtected30','ProtPerc30']

Suojeltu10 = merged_df2.Area_Protected10.sum()/merged_df2.Area_UnProtected10.sum()*100
Suojeltu30 = merged_df2.Area_Protected30.sum()/merged_df2.Area_UnProtected30.sum()*100

print(f"Zonation raja-arvo top 10%:lle on {stand_age.Zonation_mean.quantile(0.9)}")
print(f"Zonation raja-arvo top 30%:lle on {stand_age.Zonation_mean.quantile(0.7)}")
print(f"Suojellun osuus parhaasta 10 %:sta on {Suojeltu10} %")
print(f"Suojellun osuus parhaasta 30 %:sta on {Suojeltu30} %")

merged_df2[order]

stand 658767
treestand 1285332
treestandsummary 1307908
specialfeature 1307908
number of empty ages 9143
658767
groupingvariable (standid) is unique True
number of Nulls
maingroup                0
subgroup                43
fertilityclass          52
soiltype                52
drainagestate           43
ditchingyear        547281
thinningyear        658767
developmentclass     68719
standquality        658767
maintreespecies       8242
area                     0
areadecrease             0
creationtime             0
updatetime               0
Zonation_mean           70
Zonation_median         70
Zonation_min            70
Zonation_max            70
meanage               9143
protection_class         0
age_class                0
dtype: int64
Zonation raja-arvo top 10%:lle on 0.7472840740446625
Zonation raja-arvo top 30%:lle on 0.5625051836439352
Suojellun osuus parhaasta 10 %:sta on 44.09358790128449 %
Suojellun osuus parhaasta 30 %:sta on 17.316072330659406 %


,Area_Protected10,Area_UnProtected10,ProtPerc10,Area_Protected30,Area_UnProtected30,ProtPerc30
fertilityclass,,,,,,
1.0,526.967,1390.294,27.485408,557.123,1602.383,25.798632
2.0,1643.594,6481.980,20.227420,2404.440,13505.844,15.112490
3.0,18781.777,33453.116,35.956381,25041.627,116826.010,17.651402
4.0,6858.394,19897.604,25.633109,11915.917,94574.003,11.189714
5.0,2124.880,6435.001,24.823710,4134.367,27490.867,13.073001
6.0,45.352,232.983,16.294034,109.197,929.525,10.512630
7.0,13.148,132.772,9.010417,29.592,280.876,9.531417


## General

In [7]:
#the most occurring (mode) value within categorical variables, COI is ColumnsOfInterest
COI_categorical = [ 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('fertilityclass')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='fertilityclass')

#mean values for continuous variable
COI_continuous = ['fertilityclass','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('fertilityclass')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['fertilityclass','area']
grouped = stand_age[COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)

merged_df

,maingroup,subgroup,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
fertilityclass,,,,,,,,,,,,,,
1.0,1,1.0,20.0,04,2.0,0.793607,0.853636,0.832437,0.688889,0.905706,71.557634,0.218110,3026,2401.454
2.0,1,1.0,20.0,03,2.0,1.005798,0.590296,0.570212,0.435534,0.704984,57.310326,0.114036,27500,27659.447
3.0,1,1.0,10.0,03,1.0,1.547536,0.455625,0.444947,0.345464,0.572309,55.823550,0.070742,284882,440865.137
4.0,1,1.0,10.0,03,1.0,1.626456,0.423953,0.414248,0.320860,0.538073,55.974620,0.059683,257276,418448.155
5.0,1,3.0,60.0,02,1.0,1.723862,0.436207,0.426843,0.327994,0.554487,54.600764,0.082722,80922,139498.326
6.0,3,3.0,60.0,02,1.0,2.667440,0.371068,0.361574,0.269289,0.501151,49.819822,0.062724,4464,11907.452
7.0,2,1.0,50.0,NaN,1.0,0.899645,0.538512,0.519716,0.418518,0.640542,95.918699,0.057543,643,578.472
8.0,2,1.0,30.0,NaN,2.0,2.368000,0.435386,0.435386,0.318274,0.552498,44.000000,0.000000,2,4.736


In [8]:
COI_categorical = ['age_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('age_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='age_class')

#mean values for continuous variable
COI_continuous = ['age_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('age_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['age_class','area']
grouped = stand_age[COI_continuous_sum].groupby('age_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='age_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='age_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
age_class,,,,,,,,,,,,,,,
0,1,1.0,3.0,60.0,A0,1.0,1.782936,0.392136,0.385743,0.287819,0.522805,NaN,0.092967,9143,16301.381
1,1,1.0,3.0,10.0,T2,1.0,1.771272,0.360080,0.353151,0.273872,0.483865,22.374426,0.051375,156458,277129.652
2,1,1.0,3.0,10.0,03,1.0,1.519247,0.475843,0.464111,0.359504,0.588622,65.512456,0.075801,487724,740973.343
3,1,1.0,3.0,10.0,04,1.0,1.323119,0.554687,0.535727,0.412585,0.659687,154.943771,0.088938,5442,7200.416


In [9]:
#the most occurring (mode) value within categorical variables
COI_categorical = ['protection_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('protection_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='protection_class')

#mean values for continuous variable
COI_continuous = ['protection_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage']
grouped = stand_age[COI_continuous].groupby('protection_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['protection_class','area']
grouped = stand_age[COI_continuous_sum].groupby('protection_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='protection_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='protection_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,size,area_sum
protection_class,,,,,,,,,,,,,,
0,1,1.0,3.0,10.0,03,1.0,1.571099,0.432612,0.421570,0.324539,0.548668,55.255368,612425,962180.319
1,1,1.0,3.0,10.0,03,1.0,1.713877,0.649078,0.644652,0.524588,0.758319,64.062319,46342,79424.473


# Zonation 6

## Protection distribution

In [11]:
import geopandas as gpd
import pandas as pd
import numpy as np
import dask_geopandas as dask_gpd
from scipy.spatial import cKDTree
from rasterstats import zonal_stats
import rasterio
from scipy import stats
import fiona

#to see numbers as non scientific
np.set_printoptions(suppress=True)

filenameZonation = r"YuorPathToFileWithZonationValues"

filename = r"YourPath\MV_Kainuu.gpkg"

#stand polygons
stand = dask_gpd.read_file(filenameZonation, npartitions=2)
stand = stand.compute()
print("stand",len(stand))
#standids to add age etc
treestand = dask_gpd.read_file(filename, layer='treestand', npartitions=2)
treestand = treestand.compute()
condition = treestand.type.isin([2,1])
treestand = treestand[condition]
print("treestand",len(treestand))
#wanted data to add
treestandsummary = dask_gpd.read_file(filename,layer='treestandsummary', npartitions=2)
treestandsummary = treestandsummary.compute()
print("treestandsummary",len(treestandsummary))
#wanted data to add
specialfeature = dask_gpd.read_file(filename,layer='specialfeature', npartitions=2)
specialfeature = specialfeature.compute()
print("specialfeature",len(treestandsummary))
columns = ['standid', 'featurecode']
featurecode = specialfeature[columns]
featurecode = featurecode.groupby('standid').min()
featurecode.reset_index(inplace=True)
 

#add standid to treestandsummaries and join to stand polygons based on that
joined_layer = treestandsummary.merge(treestand, left_on='treestandid', right_on='treestandid', how='left')
joined_layer = stand.merge(joined_layer, left_on='standid', right_on='standid', how='left')
len(joined_layer)

#select the columns to add+join key
columns = ['standid', 'meanage']
age = joined_layer[columns]

#one polygon includes many rows of subtable. add aggregation method, here its max
#select the larger one of inventointi- and laskenta-tyyppi
max_age = age.groupby('standid').max()
max_age.reset_index(inplace=True)
#print(max_age)

#add the desired column
stand_age = stand.merge(max_age, left_on='standid', right_on='standid', how='left')
print("number of empty ages",stand_age.meanage.isna().sum())
stand_age = stand_age.merge(featurecode, left_on='standid', right_on='standid', how='left')
print(len(stand_age))

#divide into age classes
conditions = [
    (stand_age['meanage'] < 40),
    (stand_age['meanage'] >= 40) & (stand_age['meanage'] <= 140),
    (stand_age['meanage'] > 140)
]
choices = [1,2,3]
stand_age['age_class'] = np.select(conditions, choices, default=0) #0 when nodata


out_file = r"YourPath"
combined = dask_gpd.read_parquet(out_file, npartitions=2)
combined = combined.compute()

joined = gpd.sjoin(stand_age, combined)
# Count the number of points in each polygon. use an unique id column to group
print(f"groupingvariable (standid) is unique {stand_age.standid.is_unique}")
counts = joined.groupby('standid').size()

# Convert the counts Series to a DataFrame
counts_df = counts.reset_index()
counts_df.columns = ['standid', 'count']

# Merge the counts with the original GeoDataFrame
stand_age = pd.merge(stand_age, counts_df, on='standid', how='left')

# Fill NaN values with 0 (assuming no overlap means a count of 0)
stand_age['protected'] = stand_age['count'].fillna(0)

#divide into protected classes
stand_age['protection_class']  = np.where(stand_age['protected']  > 0, 1, 0)


COI = [ 'maingroup', 'subgroup', 'fertilityclass',
       'soiltype', 'drainagestate', 'ditchingyear', 'thinningyear',
       'developmentclass', 'standquality', 'maintreespecies', 'area',
       'areadecrease', 'creationtime', 'updatetime', 'Zonation_mean', 'Zonation_median',
        'Zonation_min', 'Zonation_max',
        'meanage', 'protection_class','age_class']

print("number of Nulls")
print(stand_age[COI].isna().sum())

COI_continuous_sum = ['fertilityclass','area']
condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result = grouped.sum()
result = result.rename(columns={'area': 'Area_Protected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.9)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result2 = grouped.sum()
result2 = result2.rename(columns={'area': 'Area_UnProtected10'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 1)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'Area_Protected30'})

condition = (stand_age.Zonation_mean > stand_age.Zonation_mean.quantile(0.7)) & (stand_age.protection_class == 0)
grouped = stand_age[condition][COI_continuous_sum].groupby('fertilityclass')
result4 = grouped.sum()
result4 = result4.rename(columns={'area': 'Area_UnProtected30'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result4, left_on='fertilityclass', right_index=True)


# As percentages
percentage_df = merged_df.apply(lambda x: x / x.sum() * 100)
percentage_df = percentage_df.rename(columns={'Area_Protected10': 'AP10_perc', 'Area_UnProtected10': 'AU10_perc', 'Area_Protected30': 'AP30_perc', 'Area_UnProtected30': 'AU30_perc'})

# combne with areas
merged_df2 = merged_df.merge(percentage_df, left_on='fertilityclass', right_index=True)

merged_df2['ProtPerc10'] = merged_df2.apply(lambda row: (row['Area_Protected10'] / (row['Area_UnProtected10']+row['Area_Protected10'])) * 100, axis=1)
merged_df2['ProtPerc30'] = merged_df2.apply(lambda row: (row['Area_Protected30'] / (row['Area_UnProtected30']+row['Area_Protected30'])) * 100, axis=1)
order = ['Area_Protected10', 'Area_UnProtected10', 'ProtPerc10','Area_Protected30','Area_UnProtected30','ProtPerc30']

Suojeltu10 = merged_df2.Area_Protected10.sum()/merged_df2.Area_UnProtected10.sum()*100
Suojeltu30 = merged_df2.Area_Protected30.sum()/merged_df2.Area_UnProtected30.sum()*100

print(f"Zonation raja-arvo top 10%:lle on {stand_age.Zonation_mean.quantile(0.9)}")
print(f"Zonation raja-arvo top 30%:lle on {stand_age.Zonation_mean.quantile(0.7)}")
print(f"Suojellun osuus parhaasta 10 %:sta on {Suojeltu10} %")
print(f"Suojellun osuus parhaasta 30 %:sta on {Suojeltu30} %")

merged_df2[order]

stand 658767
treestand 1285332
treestandsummary 1307908
specialfeature 1307908
number of empty ages 9143
658767
groupingvariable (standid) is unique True
number of Nulls
maingroup                0
subgroup                43
fertilityclass          52
soiltype                52
drainagestate           43
ditchingyear        547281
thinningyear        658767
developmentclass     68719
standquality        658767
maintreespecies       8242
area                     0
areadecrease             0
creationtime             0
updatetime               0
Zonation_mean           69
Zonation_median         69
Zonation_min            69
Zonation_max            69
meanage               9143
protection_class         0
age_class                0
dtype: int64
Zonation raja-arvo top 10%:lle on 0.7955826770652751
Zonation raja-arvo top 30%:lle on 0.5536760658025741
Suojellun osuus parhaasta 10 %:sta on 62.674524262893094 %
Suojellun osuus parhaasta 30 %:sta on 22.415433441785172 %


,Area_Protected10,Area_UnProtected10,ProtPerc10,Area_Protected30,Area_UnProtected30,ProtPerc30
fertilityclass,,,,,,
1.0,528.410,1243.867,29.815317,563.082,1590.561,26.145559
2.0,1739.808,5302.904,24.703665,2644.292,13010.534,16.891226
3.0,21533.786,29218.087,42.429539,29493.609,118693.421,19.902962
4.0,9124.313,16231.660,35.984866,15981.489,87016.270,15.516346
5.0,2909.687,4918.364,37.170006,6071.889,23411.107,20.594545
6.0,49.051,279.681,14.921273,146.584,1030.071,12.457687
7.0,25.193,101.840,19.831855,32.163,316.275,9.230624


## General

In [12]:
#the most occurring (mode) value within categorical variables, COI is ColumnsOfInterest
COI_categorical = [ 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('fertilityclass')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='fertilityclass')

#mean values for continuous variable
COI_continuous = ['fertilityclass','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('fertilityclass')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['fertilityclass','area']
grouped = stand_age[COI_continuous_sum].groupby('fertilityclass')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='fertilityclass', right_index=True)
merged_df = merged_df.merge(result3, left_on='fertilityclass', right_index=True)

merged_df

,maingroup,subgroup,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
fertilityclass,,,,,,,,,,,,,,
1.0,1,1.0,20.0,04,2.0,0.793607,0.837160,0.826793,0.748813,0.878817,71.557634,0.218110,3026,2401.454
2.0,1,1.0,20.0,03,2.0,1.005798,0.584985,0.572977,0.476291,0.667865,57.310326,0.114036,27500,27659.447
3.0,1,1.0,10.0,03,1.0,1.547536,0.453952,0.447278,0.371565,0.537168,55.823550,0.070742,284882,440865.137
4.0,1,1.0,10.0,03,1.0,1.626456,0.415136,0.409308,0.337494,0.496565,55.974620,0.059683,257276,418448.155
5.0,1,3.0,60.0,02,1.0,1.723862,0.421365,0.416949,0.342177,0.507575,54.600764,0.082722,80922,139498.326
6.0,3,3.0,60.0,02,1.0,2.667440,0.361899,0.357596,0.283745,0.458827,49.819822,0.062724,4464,11907.452
7.0,2,1.0,50.0,NaN,1.0,0.899645,0.579044,0.567758,0.482343,0.653209,95.918699,0.057543,643,578.472
8.0,2,1.0,30.0,NaN,2.0,2.368000,0.828264,0.828264,0.759801,0.896727,44.000000,0.000000,2,4.736


In [13]:
COI_categorical = ['age_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('age_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='age_class')

#mean values for continuous variable
COI_continuous = ['age_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage', 'protection_class']
grouped = stand_age[COI_continuous].groupby('age_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['age_class','area']
grouped = stand_age[COI_continuous_sum].groupby('age_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='age_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='age_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,protection_class,size,area_sum
age_class,,,,,,,,,,,,,,,
0,1,1.0,3.0,60.0,A0,1.0,1.782936,0.405685,0.403516,0.324545,0.501984,NaN,0.092967,9143,16301.381
1,1,1.0,3.0,10.0,T2,1.0,1.771272,0.366859,0.363538,0.300838,0.457709,22.374426,0.051375,156458,277129.652
2,1,1.0,3.0,10.0,03,1.0,1.519247,0.464657,0.457401,0.378792,0.544661,65.512456,0.075801,487724,740973.343
3,1,1.0,3.0,10.0,04,1.0,1.323119,0.576644,0.564111,0.470544,0.651924,154.943771,0.088938,5442,7200.416


In [14]:
#the most occurring (mode) value within categorical variables
COI_categorical = ['protection_class', 'maingroup', 'subgroup', 'fertilityclass','soiltype', 'developmentclass', 'maintreespecies']
grouped = stand_age[COI_categorical].groupby('protection_class')
result = grouped.apply(lambda x: x.mode().iloc[0])
result = result.drop(columns='protection_class')

#mean values for continuous variable
COI_continuous = ['protection_class','area','Zonation_mean', 'Zonation_median','Zonation_min', 'Zonation_max','meanage']
grouped = stand_age[COI_continuous].groupby('protection_class')
result2 = grouped.mean().assign(size=grouped.size())

COI_continuous_sum = ['protection_class','area']
grouped = stand_age[COI_continuous_sum].groupby('protection_class')
result3 = grouped.sum()
result3 = result3.rename(columns={'area': 'area_sum'})


#combine most occurring categorical values with mean continuous values
merged_df = result.merge(result2, left_on='protection_class', right_index=True)
merged_df = merged_df.merge(result3, left_on='protection_class', right_index=True)

merged_df

,maingroup,subgroup,fertilityclass,soiltype,developmentclass,maintreespecies,area,Zonation_mean,Zonation_median,Zonation_min,Zonation_max,meanage,size,area_sum
protection_class,,,,,,,,,,,,,,
0,1,1.0,3.0,10.0,03,1.0,1.571099,0.420535,0.413906,0.339546,0.504221,55.255368,612425,962180.319
1,1,1.0,3.0,10.0,03,1.0,1.713877,0.719078,0.717213,0.634343,0.789700,64.062319,46342,79424.473
